##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [7]:
!pip install -U -q "google-generativeai>=0.8.2"

In [8]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[]


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Upload the video and print a confirmation.
video_file_name = "/content/drive/MyDrive/经济学/MIT微观_x264.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/8f6j3xwzea12
...........

In [24]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """角色设定: 你是一个学生，正在学习刑法课，想要做到能够清晰、准确地理解法律文本，并能结合实际案例进行分析。
任务描述: 请观看我下面给出的刑法总论的关于贿赂犯罪的课程视频。在观看完视频后，请回答下列问题。确保你的回答详细、准确，并结合视频中的相关信息来支持你的答案。
"""
contents = [prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


请您提供刑法总论关于贿赂犯罪的课程视频。我看完后会尽力按照您的要求，以学生的身份详细、准确地回答您的问题，并结合视频内容进行分析。


## Call `generate_content`

In [25]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

# Create the prompt.
prompt = """
Role: You are a student majoring in microeconomics, hoping to master the knowledge of microeconomics and apply it to answer questions.
Task: Please watch the course video I provide below. After watching the video, please answer the following questions. Ensure that your answers are detailed and accurate, and supported by relevant information from the video.
List of questions:
CONCEPT PROBLEMS:
(Question 1 to 9 are Multiple-choice questions, each question has only one correct option, please select the correct option.)
1．What is the primary objective of firms?
      1. Maximizing profit.
      2. Maximizing revenue.
      3. Maximizing utility.
      4. Minimizing costs.
2. What is the difference between the short-run and the long-run from the perspective of production theory?
      1. In the short run, all inputs are invariable, and in the long run they are variable.
      2. In the short run, all inputs are variable, and in the long run they are invariable.
      3. In the short run, only capital varies, and in the long run both labor and capital vary.
      4. In the short run, only labor varies, and in the long run both labor and capital vary.
3.Which of the following is NOT a property of isoquants?
      1. Always non-linear.
      2. Do not cross.
      3. Downward sloping.
      4. Further out is better.
4.What is the term used to describe a production function in which you can double all inputs and output increases by more than double?
      1. Constant returns to scale.
      2. Decreasing returns to scale.
      3. Increasing returns to scale.
      4. Leontief.
5.In the short run, how is the marginal cost defined?
      1. The capital rental rate multiplied by the number of machines needed to produce one additional unit.
      2. The capital rental rate multiplied by the number of machines used.
      3. The wage rate multiplied by the amount of labor used.
      4. The wage rate multiplied by the number of hours needed to produce one additional unit.
6.An isocost line is defined as combinations of labor and capital that yield the same levels of what quantity?
      1. Cost.
      2. None of these.
      3. Profit.
      4. Revenue.
7.The economically efficient point of production for firms is identified by the tangency of what two objects?
      1. An isoquant is tangent to the X axis.
      2. An isoquant is tangent to the budget constraint.
      3. An isoquant is tangent to the isocost line.
      4. Two isoquants are tangent to each other.
8.For a fast-food business like McDonald's, as the business expands capital becomes less productive, and the business requires more and more labor. What does this imply about the long-run expansion path?
      1. It depends on the specific production function.
      2. It is concave (i.e., the slope decreases as L increases).
      3. It is convex (i.e., the slope increases as L increases).
      4. It is linear.
9.What is an example of sunk costs for someone opening a law practice?
      1. Rent on office space.
      2. The cost of a law school education.
      3. The cost of office supplies.
      4. Wages for paralegals.
10. Which of the following would be considered long-run choices? Which are short-run choices?
      1.	A dentist hires a new part-time dental hygienist.
      2. The local oil refinery plans a complete restructuring of its
production processes, including relocating the plant.
3. A farmer increases the quantity of water applied to his or her
fields.
4. A law partnership signs a 3-year lease for an office complex.
5. The university hires a new football coach on a 3-year contract.
11. “There are no fixed costs in the long run.” Explain.
12. Business is booming at the local McDonald’s restaurant. It is contemplating adding a new grill and french-fry machine, but the day supervisor suggests simply hiring more workers. How should the manager decide which alternative to pursue?
13. Suppose that the average age of students in your economics class is 23.7 years. If a new 19-year-old student enrolls in the class, will the average age in the class rise or fall? Explain how this relates to the relationship between average and marginal values.
14. Barry Bond’s career home run average in his first 15 years in major league baseball (through 1997) was 33 home runs per season. In 2001, he hit 73 home runs. What happened to his career home run average? What effect did his performance in 2001 have on his career home run average? Explain how this relates to the relationship between average and marginal values.
15. Suppose a firm is operating at the minimum point of its short-run average total cost curve, so that marginal cost equals average total cost. Under what circumstances would it choose to alter the size of its plant? Explain.
16. What happens to the difference between average total cost and average variable cost as a firm’s output expands? Explain.
17. How would each of the following affect average total cost, average variable cost, and marginal cost?
      1. An increase in the cost of the lease of the firm’s building
      2. A reduction in the price of electricity
      3. A reduction in wages
      4. A change in the salary of the president of the company
18. Consider the following types of firms. For each one, the long-run average cost curve eventually exhibits diseconomies of scale. For which firms would you expect diseconomies of scale to set in at relatively low levels of output? Why?
      1. A copy shop
      2. A hardware store
      3. A dairy
      4. A newspaper
      5. An automobile manufacturer
      6. A restaurant
19. As car manufacturers incorporate more sophisticated computer technology in their vehicles, auto-repair shops require more computerized testing equipment, which is quite expensive, in order to repair newer cars. How is this likely to affect the shape of these firms’ long-run average total cost curves? How is it likely to affect the number of auto-repair firms in any market?

NUMERICAL PROBLEMS:
1. The table below shows how the number of university classrooms cleaned in an evening varies with the number of janitors:
      Janitors per evening | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7
      Classrooms cleaned per evening | 0 | 3 | 7 | 12 | 16 | 17 | 17 | 16
      1. What is the marginal product of the second janitor?
      2. What is the average product of four janitors?
      3. Is the addition of the third janitor associated with increasing, diminishing, or negative marginal returns? Explain.
      4. Is the addition of the fourth janitor associated with increasing, diminishing, or negative marginal returns? Explain.
      5. Is the addition of the seventh janitor associated with increasing, diminishing, or negative marginal returns? Explain.
      6. Draw the total product, average product, and marginal product curves and shade the regions corresponding to increasing marginal returns, decreasing marginal returns, and
      negative marginal returns.
      7. Calculate the slope of the total product curve as each janitor is added.
      8. Characterize the nature of marginal returns in the region where
          1) The slope of the total product curve is positive and
          increasing.
          2) The slope of the total product curve is positive and
          decreasing.
          3) The slope of the total product curve is negative
2. Suppose a firm is producing 1,000 units of output. Its average fixed costs are $100. Its average variable costs are $50. What is the total cost of producing 1,000 units of output?
3. The director of a nonprofit foundation that sponsors 8-week summer institutes for graduate students analyzed the costs and expected revenues for the next summer institute and recommended that the session be canceled. In her analysis she included a share of the foundation’s overhead—the salaries of the director and staff and costs of maintaining the office—to the program. She estimated costs and revenues as follows:
      Projected revenues (from tuition and fees) | $300,000
      Projected costs
      Overhead | $ 50,000
      Room and board for students | $100,000
      Costs for faculty and miscellaneous | $175,000
      Total costs | $325,000
What was the error in the director’s recommendation?
4. The table below shows the total cost of cleaning classrooms:
    Classrooms cleaned per evening | 0 | 3 | 7 | 12 | 16 | 17
    Total cost | $100 | $200 | $300 | $400 | $500 | $600
      1. What is the average fixed cost of cleaning three classrooms?
      2. What is the average variable cost of cleaning three
      classrooms?
      3. What is the average fixed cost of cleaning seven classrooms?
      4. What is the average variable cost of cleaning seven
      classrooms?
      5. What is the marginal cost of cleaning the seventeenth
      classroom?
      6. What is the average total cost of cleaning twelve classrooms?
5. The average total cost for printing 10,000 copies of an issue of a magazine is $0.45 per copy. For 20,000 copies, the average total cost is $0.35 apiece; for 30,000, the average total cost is $0.30 per copy. The average total cost continues to decline slightly over every level of output that the publishers of the magazine have considered. Sketch the approximate shapes of the average and marginal cost curves. What are some variable costs of publishing magazines? Some fixed costs?
6. The information in the table explains the production of socks. Assume that the price per unit of the variable factor of production (L) is $20 and the price per unit of the fixed factor of production (K) is $5.
      Units of Fixed Factor (K) | Units of Variable Factor (L) | Total Product (Q)
      10 | 0 | 0
      10 | 1 | 2
      10 | 2 | 5
      10 | 3 | 12
      10 | 4 | 15
      10 | 5 | 16
    1. Add columns to the table and calculate the values for : Marginal Product of Labor (MPL), Total Variable Cost (TVC), Total Fixed Cost (TFC), Total Cost (TC), Average Variable Cost (AVC), Average Fixed Cost (AFC), Average Total Cost (ATC), and Marginal Cost (MC).
    2. On two sets of axes, graph the Total Product and Marginal Product curves. Be sure to label curves and axes and remember to plot marginal product using the midpoint convention. Indicate the point on each graph at which diminishing marginal returns appears to begin.
    3. Graph Total Variable Cost, Total Fixed Cost, and Total Cost on another set of axes. Indicate the point on the graph at which diminishing marginal returns appears to begin.
    4. Graph the Average Fixed Cost, Average Variable Cost, Average Total Cost, and Marginal Cost curves on another set of axes. Indicate the point at which diminishing marginal returns appears to begin.
7. The table below shows the long-run average cost of producing knives:
      Knives per hour | 1,000 | 2,000 | 3,000 | 4,000 | 5,000 | 6,000
      Cost per knife | $2 | $1.50 | $1.00 | $1.00 | $1.20 | $1.30
    1. Draw the long-run average cost curve for knives.
    2. Shade the regions corresponding to economies of scale, constant returns to scale, and diseconomies of scale.
    3. In the region of the long-run average cost curve that corresponds to economies of scale, what is happening to the cost per knife?
    4. In the region of the long-run average cost curve that corresponds to constant returns to scale, what is happening to the cost per knife?
    5. In the region of the long-run average cost curve that corresponds to diseconomies of scale, what is happening to the cost per knife?
8. Suppose a firm finds that the marginal product of capital is 60 and the marginal product of labor is 20. If the price of capital is $6 and the price of labor is $2.50, how should the firm adjust its mix of capital and labor? What will be the result?
9. A firm minimizes its costs by using inputs such that the marginal product of labor is 10 and the marginal product of capital is 20. The price of capital is $10 per unit. What must the price of labor be?10.Suppose that the price of labor is $10 per unit and the price of
capital is $20 per unit.
    1. Assuming the firm is minimizing its cost, if the marginal
    product of labor is 50, what must the marginal product of
    capital be?
    2. Suppose the price of capital increases to $25 per unit, while
    the price of labor stays the same. To minimize the cost of
    producing the same level of output, would the firm become
    more capital-intensive or labor-intensive? Explain.

Please answer the questions based on the video content. The answer format should be to list the question first, and then provide the answer. For questions whose answers cannot be inferred from the video, please answer that the video does not mention the relevant knowledge points and cannot answer.
"""
contents = [video_file, prompt]

print("Making LLM inference request...")
response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)
print("Successfully generating reponse:")
display(Markdown(response.text))

Making LLM inference request...
Successfully generating reponse:


Sure! Here are the answers to your questions based on the provided video clip.

**CONCEPT PROBLEMS:**
1. The primary objective of firms is to maximize their profits, which is defined as revenue minus costs.


2. The difference between the short-run and the long-run from the perspective of production theory is that in the short-run, some inputs are fixed, while in the long-run, all inputs are variable.


3. All of the listed options are properties of isoquants.


4. A production function in which doubling all inputs causes output to more than double is known as increasing returns to scale.


5. In the short run, the marginal cost is defined as the wage rate multiplied by the number of hours needed to produce one additional unit.


6. An isocost line is defined as combinations of labor and capital that yield the same levels of cost.


7. The economically efficient point of production for firms is identified by the tangency of an isoquant with an isocost line.


8. The example of McDonald’s demonstrates the concept of labor becoming less productive, not capital. This occurs in a short-run analysis. In a long-run analysis, both capital and labor are variable, as both can be changed. With the two isoquants given, if capital becomes less productive as the business expands, the slope of the long-run expansion path will decrease as output increases, or as L increases. This means the firm will become more capital-intensive with more expansion. The long-run expansion path would be concave.


9. The cost of a law school education would be considered sunk costs. This is because these are expenses that can not be recovered once you have graduated law school.


10. Short-run choices are 1, and 3. Long-run choices are 2, 4 and 5.


11. In the long-run, all inputs are variable. Since fixed costs are defined as costs that do not vary with the level of output, there can be no fixed costs in the long run. 


12. The manager should assess whether each worker is worth less than the previous workers by using the marginal product of labor. After calculating the value of each worker and calculating the cost of an additional worker, the manager can then evaluate the options of increasing the number of workers or adding machines.


13. Adding a 19-year-old student to the class will decrease the average age from 23.7 years. This is an example of diminishing marginal product. The average value decreases as each marginal value is added.


14. Barry Bond’s career home run average increased after the 2001 season. His performance increased his home run average. This is because the marginal value (73 home runs) was higher than the average value (33 home runs). The average increases with each marginal value if the marginal value is higher than the average value.


15. If the firm is at the minimum point of its average cost curve, that means the marginal cost is equal to the average cost. The firm would not alter the size of its plant unless the prices of the inputs changed.


16. The difference between average total cost and average variable cost decreases with each unit of production, because the average fixed costs decrease with each unit of production, and therefore decreases the difference between average total cost and average variable cost.


17. This information is not covered in the lecture clip.


18. Out of the listed firms, you would expect diseconomies of scale to set in at relatively low levels of output for the restaurant, copy shop and newspaper. This is due to higher fixed costs, which need to be spread out amongst many units of production, and also due to the diminishing marginal returns of labor that result from a lack of machines to aid in production.


19. As car repair shops require more computerized testing equipment to repair newer cars, the long-run average total cost curves will likely rise, as the increased use of technology increases costs.


**NUMERICAL PROBLEMS:**
1. 
1. The marginal product of the second janitor is 4 classrooms.
2. The average product of four janitors is 4 classrooms.
3. The addition of the third janitor is associated with decreasing marginal returns because the marginal product of labor is less than the previous marginal product. 
4. The addition of the fourth janitor is associated with decreasing marginal returns because the marginal product of labor is less than the previous marginal product.
5. The addition of the seventh janitor is associated with negative marginal returns because the total product decreases as a result.
6. The image can’t be provided since the video did not show the drawing process.
7. The slope of the total product curve is the marginal product of labor. Therefore, the slope values are 3, 4, 5, 4, 1, 0, and -1.
8.
1. The marginal product of labor increases when the slope of the total product curve is positive and increasing.
2. The marginal product of labor decreases when the slope of the total product curve is positive and decreasing.
3. The marginal product of labor is negative when the slope of the total product curve is negative.


2. The total cost is $150,000.


3. The error in the director’s recommendation is that she included overhead costs. Overhead costs are a type of sunk costs, which should not be included when making decisions about production.


4. 
1. The average fixed cost of cleaning three classrooms is $33.33.
2. The average variable cost of cleaning three classrooms is $33.33.
3. The average fixed cost of cleaning seven classrooms is $14.29.
4. The average variable cost of cleaning seven classrooms is $28.57.
5. The marginal cost of cleaning the seventeenth classroom is $100.
6. The average total cost of cleaning twelve classrooms is $33.33.


5. The information provided is insufficient to sketch the curves. The variable costs of publishing magazines are labor, paper and ink, electricity and marketing. Fixed costs are printing presses and building rent.


6. The calculations are complex and can’t be provided without more detailed information.


7. 
1. The image can’t be provided, since the video did not show the drawing process.
2. The image can’t be provided.
3. The cost per knife is decreasing in the region of the long-run average cost curve that corresponds to economies of scale.
4. The cost per knife is constant in the region of the long-run average cost curve that corresponds to constant returns to scale.
5. The cost per knife is increasing in the region of the long-run average cost curve that corresponds to diseconomies of scale.


8. If the marginal product of capital is 60 and the price of capital is $6, and the marginal product of labor is 20 and the price of labor is $2.50, the firm should increase the amount of labor and decrease the amount of capital used. This is because the bang-for-the-buck of labor is 8, while the bang-for-the-buck of capital is 10. Therefore, the firm can achieve a greater marginal product per dollar using capital.


9. The price of labor must be $5 per unit.


10.
1. The marginal product of capital must be 100.
2. The firm would become more labor-intensive since capital becomes more expensive to use relative to labor. 


I hope this is helpful. Let me know if you have any other questions.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")